In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pyaudio

%matplotlib tk

# 设置音频参数
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 48000
CHUNK = 2048
trigger_frequency = 500
cnt = 0

# 设置要分析的四个特定频率
target_frequencies = [697, 770, 852, 941, 1209, 1336, 1477, 1633, trigger_frequency]
frequency_range = 20  # 频率范围

# 计算频率范围的边界
frequency_bounds = [(f - frequency_range/2, f + frequency_range/2) for f in target_frequencies]
# 创建用于绘图的图表和子图
fig, ax = plt.subplots()
bars = ax.bar(range(len(target_frequencies)), np.zeros(len(target_frequencies)))

# 设置y轴范围
ax.set_ylim(4, 7)  # 替换为适当的值

# 设置x轴刻度和标签
ax.set_xticks(range(len(target_frequencies)))
ax.set_xticklabels([f"{freq} Hz" for freq in target_frequencies])

# 定义DTMF频率对应表
dtmf_freqs = {
    '0': [(697, 1209), (1209, 697)],
    '1': [(697, 1336), (1336, 697)],
    '2': [(697, 1477), (1477, 697)],
    '3': [(697, 1633), (1633, 697)],
    '4': [(770, 1209), (1209, 770)],
    '5': [(770, 1336), (1336, 770)],
    '6': [(770, 1477), (1477, 770)],
    '7': [(770, 1633), (1633, 770)],
    '8': [(852, 1209), (1209, 852)],
    '9': [(852, 1336), (1336, 852)],
    '10': [(852, 1477), (1477, 852)],
    '11': [(852, 1633), (1633, 852)],
    '12': [(941, 1209), (1209, 941)],
    '13': [(941, 1336), (1336, 941)],
    '14': [(941, 1477), (1477, 941)],
    '15': [(941, 1633), (1633, 941)],
}

# Set the threshold for starting and stopping sampling
start_amplitude_threshold = 6.5
stop_amplitude_threshold = 6.5

# Initialize variables for sampling
is_sampling = False
samples = []
current_sample = []
result = []

# Define a function to convert frequency pairs to characters
def freqs_to_char(freq_pair):
    for char, freq_pairs in dtmf_freqs.items():
        if freq_pair in freq_pairs:
            return char
    return None

# 初始化音频输入
p = pyaudio.PyAudio()
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

# 更新图表并进行信号识别
def update(frame):
    global cnt, is_sampling, current_sample, samples
    
    try:
        # Read audio data
        data = np.frombuffer(stream.read(CHUNK), dtype=np.int16)
        
        fft_result = np.fft.fft(data)
        frequencies_fft = np.fft.fftfreq(len(fft_result), 1/RATE)
        
        # Calculate magnitudes
        magnitudes = [np.mean(np.abs(fft_result[(frequencies_fft >= f_low) & (frequencies_fft <= f_high)]))
                      for (f_low, f_high) in frequency_bounds]
        trigger_amplitude = np.log10(magnitudes[8]+1)

        # Check if the amplitude exceeds the start threshold
        if not is_sampling and trigger_amplitude > start_amplitude_threshold:
            is_sampling = True
            current_sample = []

        # Check if the amplitude falls below the stop threshold
        elif is_sampling and trigger_amplitude < stop_amplitude_threshold:
            is_sampling = False
            if current_sample:
                # Convert frequency pairs to characters
                chars = [freqs_to_char(freq_pair) for freq_pair in current_sample]
                # Filter out None values 
                chars = [char for char in chars if char is not None]
                # Store the characters in the samples array
                samples.extend(chars)

                # Find the most common character in the samples array
                most_common_char = max(set(samples), key=samples.count)

                # Output the most common character
                print(f"Detected character: {most_common_char}")
                samples.clear()

        # If sampling, record the current frequency pair
        if is_sampling:
            # Extract the frequencies at the trigger index
            threshold = 6.5  
            if any(np.log10(m + 1) > threshold for m in magnitudes[:8]):
                # Find the indices of the two maximum magnitudes
                max_indices = np.argsort(magnitudes[:8])[-2:]

                # Get the corresponding frequencies
                freq1 = target_frequencies[max_indices[0]]
                freq2 = target_frequencies[max_indices[1]] 
                freq_pair = (freq1, freq2)
                
                # Output the frequencies
                # print(freqs_to_char(freq_pair))
                result.append(freqs_to_char(freq_pair))
                
                # Add the frequency pair to the current sample
                current_sample.append(freq_pair)

        # Update the bar chart
        for bar, magnitude in zip(bars, magnitudes):
            magnitude_log = np.log10(magnitude + 1)
            bar.set_height(magnitude_log)
        
        return bars
    except Exception as e:
        print(f"Error: {e}")

# 设置动画更新间隔
ani = animation.FuncAnimation(fig, update, blit=False, interval=10)

# 显示图表
plt.show(block=True)

# 关闭音频流和 PyAudio 对象
stream.stop_stream()
stream.close()
p.terminate()

C:\Users\Tom_yang\AppData\Local\Temp\ipykernel_6912\3235376116.py:147: UserWarning: frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.
  ani = animation.FuncAnimation(fig, update, blit=False, interval=10)


Detected character: 9
Detected character: 7
Detected character: 12
Detected character: 13
Detected character: 13
Detected character: 13
Detected character: 10
Detected character: 7
Detected character: 9
Detected character: 7
Detected character: 3
Detected character: 6
Detected character: 8
Detected character: 13
Detected character: 10
Detected character: 7
Detected character: 12
Detected character: 13
Detected character: 12
Detected character: 12
Detected character: 13
Detected character: 13
Detected character: 12
Detected character: 9
Detected character: 9
Detected character: 3
Detected character: 8
Detected character: 13
Detected character: 15
Detected character: 6
Detected character: 12
Detected character: 13
Detected character: 11
Detected character: 12
Detected character: 12
Detected character: 13
Detected character: 13
Detected character: 11
Detected character: 11
Detected character: 8
Detected character: 8
Detected character: 10
Detected character: 15
Detected character: 6
Detec